In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Parse and Chunk sectional contents from CSA and SIP documents

In [3]:
pip install pymupdf pandas langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.7 MB/s eta 0:00:00


In [4]:
import os
import re
import fitz  # PyMuPDF
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configuration
PDF_DIRECTORY = "/content/drive/MyDrive/LLM/CalWorks/Vector Database/PDFs"  # Set this to your PDF folder path

county_names = [
    "Alameda", "Alpine", "Amador", "Butte", "Calaveras", "Colusa", "Contra Costa",
    "Del Norte", "El Dorado", "Fresno", "Glenn", "Humboldt", "Imperial", "Inyo",
    "Kern", "Kings", "Lake", "Lassen", "Los Angeles", "Madera", "Marin", "Mariposa",
    "Mendocino", "Merced", "Modoc", "Mono", "Monterey", "Napa", "Nevada", "Orange",
    "Placer", "Plumas", "Riverside", "Sacramento", "San Benito", "San Bernardino",
    "San Diego", "San Francisco", "San Joaquin", "San Luis Obispo", "San Mateo",
    "Santa Barbara", "Santa Clara", "Santa Cruz", "Shasta", "Sierra", "Siskiyou",
    "Solano", "Sonoma", "Stanislaus", "Sutter", "Tehama", "Trinity", "Tulare",
    "Tuolumne", "Ventura", "Yolo", "Yuba"
]

# Aliases for counties based on common filename patterns
alias_map = {
    "icdss": "Imperial",
    "lacdss": "Los Angeles",
    "ocss": "Orange",
    "scc": "Santa Clara",
    "sbcs": "San Bernardino",
    # Add more aliases as needed
}

def infer_metadata_from_filename(filename):
    # Clean and normalize
    name_clean = filename.lower().replace("_", " ").replace("-", " ").replace(".pdf", "")
    name_compressed = re.sub(r"\s+", "", name_clean)

    # Report type logic
    if "sip" in name_clean or "system improvement" in name_clean:
        report_type = "Cal-SIP"
    elif (
        "csa" in name_clean
        or "self-assessment" in name_clean
        or "calworks self-assessment" in name_clean
        or "county self-assessment" in name_clean
        or "fatal flaw" in name_clean
    ):
        report_type = "Cal-CSA"
    else:
        report_type = "Unknown"

    # Try full county match
    normalized_counties = [c.lower().replace(" ", "") for c in county_names]
    county = "Unknown"
    for orig, compressed in zip(county_names, normalized_counties):
        if compressed in name_compressed:
            county = orig
            break

    # If not found, try alias map
    if county == "Unknown":
        for alias, full_name in alias_map.items():
            if alias in name_compressed:
                county = full_name
                break

    return {
        "file": filename,
        "county": county,
        "report_type": report_type
    }


# A Table of Contents is needed
def extract_sections_via_toc(pdf_path, county, report_type, toc_max_pages=5):
    import fitz
    import re

    doc = fitz.open(pdf_path)
    max_pages = len(doc)
    section_entries = []

    toc_lines = []
    for i in range(min(toc_max_pages, len(doc))):
        text = doc[i].get_text()
        lines = text.split("\n")
        for line in lines:
            line = line.strip()
            # Match: Section headers OR named sections like "Introduction", "Executive Summary"
            match = re.match(
                r"^((Section\s+\d+(\.\d+)?[.:]?\s+.+?)|(?:Introduction|Executive Summary|Demographics))\s+\.{3,}\s+(\d{1,3})$",
                line,
                re.IGNORECASE
            )
            if match:
                title = match.group(1).strip()
                page = int(match.group(4))
                toc_lines.append((title, page))

    # Construct section page ranges
    for i, (title, start_page) in enumerate(toc_lines):
        end_page = toc_lines[i + 1][1] - 1 if i + 1 < len(toc_lines) else max_pages
        section_entries.append({
            "county": county,
            "report_type": report_type,
            "section_header": title,
            "start_page": start_page,
            "end_page": end_page
        })

    # Extract section text
    for section in section_entries:
        start = max(0, section["start_page"] - 1)
        end = min(section["end_page"], max_pages)
        text = "".join(doc[p].get_text() for p in range(start, end))
        section["text"] = text.strip()

    return section_entries


def chunk_sections(sections, chunk_size=1000, overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    chunked = []
    for sec in sections:
        if not sec["text"].strip():
            continue
        splits = splitter.split_text(sec["text"])
        for i, chunk in enumerate(splits):
            chunked.append({
                "county": sec["county"],
                "report_type": sec["report_type"],
                "section": sec["section_header"],
                "page": sec["start_page"],
                "chunk_id": f"{sec['section_header']}_chunk{i}",
                "text": chunk
            })
    return chunked

def main():
    pdf_files = [f for f in os.listdir(PDF_DIRECTORY) if f.endswith(".pdf")]
    all_sections = []

    for file in pdf_files:
        meta = infer_metadata_from_filename(file)
        if meta["county"] == "Unknown" or meta["report_type"] == "Unknown":
            print(f"Skipping: {file} (missing county or type)")
            continue
        try:
            path = os.path.join(PDF_DIRECTORY, file)
            sections = extract_sections_via_toc(path, meta["county"], meta["report_type"])

            if len(sections) == 0:
                print(f"No TOC sections found for: {file}")
            else:
                print(f"Found {len(sections)} sections in {file}")
                all_sections.extend(sections)

        except Exception as e:
            print(f"Error processing {file}: {e}")

    chunks = chunk_sections(all_sections)
    print(f"\n{len(all_sections)} sections to {len(chunks)} chunks")

    df = pd.DataFrame(chunks)
    df.to_excel("/content/drive/MyDrive/LLM/CalWorks/Vector Database/Output/chunked_sip_csa_output.xlsx", index=False)
    print("Saved to chunked_sip_csa_output.csv")

if __name__ == "__main__":
    main()


Found 12 sections in CSA-Summary-Santa-Clara-Fatal-Flaw.pdf
Found 12 sections in CSA-Summary-Sacramento-Fatal-Flaw.pdf
Found 12 sections in CSA-Summary-San-Francisco-Fatal-Flaw.pdf
Found 12 sections in CSA-Summary-San-Mateo-Fatal-Flaw.pdf
Found 12 sections in CSA Summary_Fresno_Fatal Flaw.pdf
Found 12 sections in CSA Report_Orange.pdf
Found 12 sections in CSA-Summary-Solano-Report.pdf
Found 11 sections in CSA_Sonoma.pdf
Found 12 sections in CSA_Tulare.pdf
Found 12 sections in CSA_Santa_Cruz.pdf
Found 12 sections in CSA_San_Luis_Obispo.pdf
Found 12 sections in CSA_Contra_Costa.pdf
Found 12 sections in CSA_Yolo.pdf
Found 11 sections in CSA_Ventura.pdf
Found 12 sections in CSA-Summary-Alameda-Fatal-Flaw.pdf
Found 12 sections in CSA_San_Diego.pdf
Found 12 sections in CSA Summary-SantaBarbara.pdf
Found 12 sections in CSA-Summary-Placer.pdf
Found 12 sections in CSA_Mariposa.pdf
Found 12 sections in CSA Riversides.pdf
Found 9 sections in CSA Mendocino.pdf
Found 12 sections in CSA-Tehama.pdf
F

# ChromaDb

In [5]:
!pip install openai tiktoken chromadb sentence-transformers langchain_community
!pip install langchain_chroma -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.2 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.6 MB/s eta 0:00:00
  Attempting uninstall: chromadb
    Found existing installation: chromadb 1.0.8
    Uninstalling chromadb-1.0.8:
      Successfully uninstalled chromadb-1.0.8


In [3]:
import os
import re
import shutil
import pandas as pd
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

# Configuration
XLSX_PATH   = "/content/drive/MyDrive/LLM/CalWorks/Vector Database/Output/chunked_sip_csa_output.xlsx"
PERSIST_DIR = "/content/drive/MyDrive/LLM/CalWorks/Vector Database/Output/chroma_sip_csa_db"
COLLECTION  = "sip_csa_chunks"
OPENAI_MODEL = "text-embedding-3-small"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

assert OPENAI_API_KEY, "OPENAI_API_KEY not set in environment"


# Normalize Text
def normalize_text(text: str) -> str:
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"‐|–|—", "-", text)
    text = re.sub(r"“|”|\"|''", '"', text)
    text = re.sub(r"’|‘|`", "'", text)
    return text

# Load Excel Data
df = pd.read_excel(XLSX_PATH).dropna(subset=["text"])
df["chunk_id"] = df.apply(
    lambda row: f"{row['county'].replace(' ', '')}_{row['report_type'].replace('-', '')}_{row['section'].replace(':', '').replace('.', '').replace(' ', '')}_chunk{row.name}",
    axis=1
)
df["text"] = df["text"].apply(normalize_text)
df["section"] = df["section"].astype(str).apply(normalize_text)

# Initialize Chroma Client
chroma_client = chromadb.PersistentClient(
    path=PERSIST_DIR,
    settings=Settings(anonymized_telemetry=False)
)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_MODEL
)

collection = chroma_client.get_or_create_collection(
    name=COLLECTION,
    embedding_function=openai_ef,
    metadata={"hnsw:space": "cosine"}
)

print(f"📦 Collection ready: {COLLECTION}")
print(f"🔎 Initial document count: {collection.count()}")

# Prepare Lists
docs = [
    f"County: {row['county']}\nSection: {row['section']}\n\n{row['text']}\n\n[End of Section: {row['section']} – County: {row['county']}]"
    for _, row in df.iterrows()
]
ids   = df["chunk_id"].astype(str).tolist()
metas = df[["county", "report_type", "section", "page"]].to_dict(orient="records")

# Upload with Debug Logs
BATCH_SIZE = 100
for i in range(0, len(docs), BATCH_SIZE):
    end = min(i + BATCH_SIZE, len(docs))
    collection.add(
        documents=docs[i:end],
        ids=ids[i:end],
        metadatas=metas[i:end]
    )
    print(f"Uploaded batch {i//BATCH_SIZE + 1}: {end - i} items")

# Post-upload verification
print(f"Final document count in collection: {collection.count()}")

📦 Collection ready: sip_csa_chunks
🔎 Initial document count: 0
✅ Uploaded batch 1: 100 items
✅ Uploaded batch 2: 100 items
✅ Uploaded batch 3: 100 items
✅ Uploaded batch 4: 100 items
✅ Uploaded batch 5: 100 items
✅ Uploaded batch 6: 100 items
✅ Uploaded batch 7: 100 items
✅ Uploaded batch 8: 100 items
✅ Uploaded batch 9: 100 items
✅ Uploaded batch 10: 100 items
✅ Uploaded batch 11: 100 items
✅ Uploaded batch 12: 100 items
✅ Uploaded batch 13: 100 items
✅ Uploaded batch 14: 100 items
✅ Uploaded batch 15: 100 items
✅ Uploaded batch 16: 100 items
✅ Uploaded batch 17: 100 items
✅ Uploaded batch 18: 100 items
✅ Uploaded batch 19: 100 items
✅ Uploaded batch 20: 100 items
✅ Uploaded batch 21: 100 items
✅ Uploaded batch 22: 100 items
✅ Uploaded batch 23: 100 items
✅ Uploaded batch 24: 100 items
✅ Uploaded batch 25: 100 items
✅ Uploaded batch 26: 100 items
✅ Uploaded batch 27: 100 items
✅ Uploaded batch 28: 100 items
✅ Uploaded batch 29: 100 items
✅ Uploaded batch 30: 100 items
✅ Uploaded batch

In [4]:
# Optional: Run test query
try:
    result = collection.query(query_texts=["childcare support"], n_results=2)
    print("\nSample Query Result:")
    for i, r in enumerate(result['documents'][0]):
        print(f"Result {i+1}: {r[:300]}...\n---")
except Exception as e:
    print(f"Error during sample query: {e}")


Sample Query Result:
Result 1: County: Santa Barbara
Section: Section 4. Initial Engagement Analysis

2. Describe the process for providing a full-time child care authorization to client (including vouchers, payment, tracking, etc.). Securing reliable quality childcare is essential to participate in the Welfare to Work program. F...
---
Result 2: County: San Mateo
Section: Section 4. Initial Engagement Analysis

authorization to client (including vouchers, payment, tracking, etc.). Full-time childcare authorization is provided through the Immediate and Continuous Eligibility process. Once a family is referred to WTW, if the family has a chil...
---
